In [ ]:
import random
import gym
import numpy as np
import random
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
from sklearn.preprocessing import StandardScaler
import random
# Normalize features
from sklearn.preprocessing import StandardScaler
from collections import deque

import collections
from keras import backend as K
import tensorflow as tf
from gym import spaces
import matplotlib.pyplot as plt
K.clear_session()

In [ ]:
data = pd.read_csv('/kaggle/input/eqqerff/preprocessed.csv')
data.drop(columns= ['Unnamed: 0','index'], inplace = True)
# preprocessing
data['DATETIME'] = pd.to_datetime(data['DATETIME'])
data['TIME'] = data['TIME'].astype(str).str.rstrip('000')
data['hour'] = data['DATETIME'].dt.hour
data['minute'] = data['DATETIME'].dt.minute
data['day_of_week'] = data['DATETIME'].dt.day_name()
data['second'] =  data['DATETIME'].dt.second
day_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
data['day_of_week'] = data['day_of_week'].map(day_map)
data.drop(columns= ['DATETIME','TIME'], inplace = True) 
data.drop(columns= ['TRADE_PRICE','TRADENO'], inplace = True)
data= data.dropna()
data

In [ ]:
def reward_function(df):
    # Calculate best bid-ask spread for each date
    best_spread_per_date = df.groupby('DATE').apply(lambda x: x['BID_PRICE'].max() - x['ASK_PRICE'].min())

    rewards = []
    for index, row in df.iterrows():
        # Calculate execution price deviation from best bid-ask spread
        execution_price_deviation = row['PRICE'] - best_spread_per_date[row['DATE']]

        # Calculate execution speed penalty
        execution_speed_penalty = 100 / 1000  # converting milliseconds to seconds

        # Calculate commission cost
        commission_cost = 0.01 * row['TRADE_VOLUME']
        # Calculate market impact penalty (quadratic function)
        market_impact_penalty = 0.0001 * (row['TRADE_VOLUME'] ** 2)

        # Calculate total reward
        reward = -execution_price_deviation - execution_speed_penalty -  commission_cost- market_impact_penalty
        rewards.append(reward)

    return rewards

rewards = reward_function(data)

In [ ]:
from tqdm import tqdm
# Normalizing the data
def normalize_data(df):
    df_normalized = df.copy()
    for column in df.columns:
        if df[column].dtype == np.float64 or df[column].dtype == np.int64:
            df_normalized[column] = (df[column] - df[column].mean()) / df[column].std()
    return df_normalized

data = normalize_data(data)
data.drop(columns = ['DATE', 'ACTION','ORDERNO','TRADE_TIME'], inplace = True)
# Create a dictionary to map string values to numeric values
buysell_map = {'B': 0, 'S': 1}

# Map the string values to numeric values
data['BUYSELL'] = data['BUYSELL'].map(buysell_map)

In [ ]:
data.drop(columns = 'TRADEPRICE', inplace = True)

In [ ]:
from tqdm import tqdm
# Ensure no missing values remain in the dataset
data.fillna(0, inplace=True)

# Define state representation
def get_state(df, step):
    state = df.iloc[step].values.astype(np.float32)
    return state

# Define the Trading Environment
class TradingEnvironment:
    def __init__(self, data):
        self.data = data
        self.current_step = 0
        self.balance = 100000  # initial balance
        self.shares_held = 0
        self.initial_balance = 100000
        self.done = False

    def reset(self):
        self.current_step = 0
        self.balance = 100000
        self.shares_held = 0
        self.done = False
        return get_state(self.data, self.current_step)

    def step(self, action):
        current_data = self.data.iloc[self.current_step]
        current_price = current_data['PRICE']
        bid_price = current_data['BID_PRICE']
        ask_price = current_data['ASK_PRICE']
        bid_ask_spread = current_data['BID_ASK_SPREAD']
        market_direction = current_data['BUYSELL']

        # Execute action
        if action == 0:  # Buy
            self.shares_held += 1
            self.balance -= ask_price
        elif action == 1:  # Sell
            self.shares_held -= 1
            self.balance += bid_price

        # Calculate reward
        total_value = self.balance + self.shares_held * current_price
        pnl = total_value - self.initial_balance
        transaction_costs = abs(self.shares_held) * bid_ask_spread
        slippage = abs(current_price - (bid_price if action == 0 else ask_price))
        market_impact = abs(self.shares_held) * current_price
        market_direction_reward = 10 if (market_direction == 0 and action == 0) or (market_direction == 1 and action == 1) else -10 if (market_direction == 0 and action == 1) or (market_direction == 1 and action == 0) else 0

        reward = pnl - transaction_costs - slippage - market_impact + market_direction_reward

        # Move to the next step
        self.current_step += 1
        if self.current_step >= len(self.data) - 1:
            self.done = True

        return get_state(self.data, self.current_step), reward, self.done

    def render(self):
        pass

class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=10000)
        self.gamma = 0.99  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.batch_size = 40
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = QNetwork(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        act_values = self.model(state)
        return torch.argmax(act_values, dim=1).item()

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            next_state = torch.FloatTensor(next_state).unsqueeze(0).to(self.device)
            target = reward
            if not done:
                target = (reward + self.gamma * torch.max(self.model(next_state)).item())
            target_f = self.model(state).detach().clone()
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = self.criterion(self.model(state), target_f)
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_state_dict(torch.load(name))

    def save(self, name):
        torch.save(self.model.state_dict(), name)

# Initialize environment and agent
env = TradingEnvironment(data)
state_size = env.data.shape[1]
action_size = 3  # Buy, Sell, Hold
agent = DQNAgent(state_size, action_size)

# Training the agent
episodes = 100
rewards = []

# Initialize the plot
plt.ion()
fig, ax = plt.subplots()
line, = ax.plot(rewards)
plt.xlabel('Episode')
plt.ylabel('Total Rewards')
plt.title('Training Rewards over Episodes')

for e in tqdm(range(episodes)):
    state = env.reset()
    total_rewards = 0

    while True:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        reward = reward if not done else -10
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_rewards += reward
        if done:
            rewards.append(total_rewards)
            print(f"Episode {e+1}/{episodes} - Total reward: {total_rewards}, Epsilon: {agent.epsilon}")
            break
        agent.replay()
    
    # Update the plot every 10 episodes
    if (e + 1) % 10 == 0:
        line.set_ydata(rewards)
        line.set_xdata(range(len(rewards)))
        ax.relim()
        ax.autoscale_view()
        plt.draw()
        plt.pause(0.001)

# Evaluation Metrics
def evaluate_agent(agent, env, episodes=10):
    total_rewards = []
    for e in range(episodes):
        state = env.reset()
        total_reward = 0

        while True:
            action = np.argmax(agent.model(torch.FloatTensor(state).unsqueeze(0)).detach().numpy())
            next_state, reward, done = env.step(action)
            state = next_state
            total_reward += reward
            if done:
                total_rewards.append(total_reward)
                break

    avg_reward = np.mean(total_rewards)
    return avg_reward

average_reward = evaluate_agent(agent, env)
print(f"Average reward over {episodes} episodes: {average_reward}")

# Plotting the final rewards
plt.ioff()
plt.plot(rewards)
plt.xlabel('Episode')
plt.ylabel('Total Rewards')
plt.title('Training Rewards over Episodes')
plt.show()

In [ ]:
# Plotting the final rewards
plt.ioff()
plt.plot(rewards)
plt.xlabel('Episode')
plt.ylabel('Total Rewards')
plt.title('Training Rewards over Episodes')
plt.show()

# TWAP and VWAP Implementation for Comparison
def TWAP(data):
    shares_to_trade = 100  # Total shares to trade
    steps = len(data)
    shares_per_step = shares_to_trade // steps
    total_value = 0
    for i in range(steps):
        price = data.iloc[i]['PRICE']
        total_value += shares_per_step * price
    return total_value / shares_to_trade

def VWAP(data):
    total_value = 0
    total_volume = 0
    for i in range(len(data)):
        price = data.iloc[i]['PRICE']
        volume = data.iloc[i]['VOLUME']
        total_value += price * volume
        total_volume += volume
    return total_value / total_volume

twap_result = TWAP(data)
vwap_result = VWAP(data)

print(f"TWAP Result: {twap_result}")
print(f"VWAP Result: {vwap_result}")

# Heatmap for Strategy Visualization
action_matrix = np.zeros((episodes, len(data)))
for i in range(episodes):
    for j, action in enumerate(actions[i]):
        action_matrix[i, j] = action

plt.figure(figsize=(10, 8))
sns.heatmap(action_matrix, cmap="YlGnBu", cbar=True)
plt.xlabel('Time Step')
plt.ylabel('Episode')
plt.title('Heatmap of Agent Actions Over Episodes')
plt.show()